# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.



## Откройте таблицу и изучите общую информацию о данных

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; <style>}"))

In [2]:
import pandas as pd

try:
    data = pd.read_csv('E:/BuduBogatym/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [15]:
data.duplicated().sum()

54

In [16]:
data = data.drop_duplicates()

In [17]:
data['education'] = data['education'].str.lower()

### Категоризация данных

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследование данных

In [23]:
def counts_children(children):# Функция для определения наличия детей
    if children > 0:
        return 'есть дети'
    else:
        return 'нет детей'
    
data['having_children'] = data['children'].apply(counts_children)
display(data.head(10))# Проверка ожидаемого результата.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,having_children
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,есть дети
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,есть дети
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,нет детей
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,есть дети
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,нет детей
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью,нет детей
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью,нет детей
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования,нет детей
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы,есть дети
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью,нет детей


Создадим сводную таблицу под нашу задачу:

In [24]:
data_children = data.pivot_table(index = 'having_children',values = 'debt',aggfunc = ['count','sum','mean'])

data_children['debt_pers'] = data_children['mean']['debt']*100
data_children = data_children.sort_values('debt_pers',ascending=False)
display(data_children)


# debt_pers показывает процент должников в зависимости от отсутствия или наличия детей у клиента

,count,sum,mean,debt_pers
,debt,debt,debt,
having_children,,,,
есть дети,7241,669,0.092391,9.239055
нет детей,14107,1063,0.075353,7.535266


Можно заметить зависимость наличия детей на процент должников, однако для более подробной информации составим таблицу, в зависимости от числа детей

In [25]:
df_children = data.pivot_table(index = 'children',values = 'debt',aggfunc = ['count','sum','mean'])

df_children['debt_pers'] = df_children['mean']['debt']*100
data_children = df_children.sort_values('debt_pers',ascending=False)
display(df_children)

# debt_pers отражает процент должников в зависимости от количества детей у клиента

,count,sum,mean,debt_pers
,debt,debt,debt,
children,,,,
0,14107,1063,0.075353,7.535266
1,4809,444,0.092327,9.232689
2,2052,194,0.094542,9.454191
3,330,27,0.081818,8.181818
4,41,4,0.097561,9.756098
5,9,0,0.000000,0.000000


**Вывод:** В результатах сводной таблице можно отметить наименьший процент среди должников у клиентов без детей. В то время как наличие 1-2 ребенка уже внушительно сказывается на проценте должников, что не может не быть явным признаком зависимости количества детей от уплаты срока в кредит. В данном случае разница составляет около 2.7%, что должно повлиять на подбор клиентов для предоставления кредита. (Клиентов с 3-5 детьми в рамках задачи можно не учитывать так как это не отразит полноту картины из-за низкого количества людей в данных категориях)

Создадим сводную таблицу под нашу задачу:

In [26]:
data_status = data.pivot_table(index = 'family_status',values = 'debt',aggfunc = ['count','sum','mean'])

data_status['debt_pers'] = data_status['mean']['debt']*100
data_status = data_status.sort_values('debt_pers',ascending=False)
display(data_status)

# debt_pers демонстрирует процент должников в зависимости от семейного положения


,count,sum,mean,debt_pers
,debt,debt,debt,
family_status,,,,
Не женат / не замужем,2796,273,0.097639,9.763948
гражданский брак,4146,385,0.092861,9.286059
женат / замужем,12266,927,0.075575,7.557476
в разводе,1189,84,0.070648,7.064760
вдовец / вдова,951,63,0.066246,6.624606


**Вывод:** На основе сводной таблицы можно проследить зависимость процента должников от семейного положения. Однако количество разведенных и вдовцов(вдов) в сравнении с  другими категориями относительно мало, поэтому ими можно пренебречь в данном случае. Основную зависимость срока уплаты кредита от семейного положения можно проследить на людей, находящихся в браке, и одиноких. В таблице видно, что отсутствие партнёра в жизни клиента отрицательно влияет на уплату кредита в срок, в то время как наличие партнёра - наоборот. Поэтому можно сделать простой вывод, что при выдаче кредита выгоднее и надёжнее довериться женатым/замужним.

Создадим свобдную таблицу для решения нашей задачи:

In [27]:
data_income = data.pivot_table(index = 'total_income_category',values = 'debt',aggfunc = ['count','mean'])

data_income['debt_pers'] = data_income['mean']['debt']*100
data_income = data_income.sort_values('debt_pers',ascending=False)
display(data_income)

# debt_pers демонстрирует процент должников в зависимости от его дохода


,count,mean,debt_pers
,debt,debt,
total_income_category,,,
E,22,0.090909,9.090909
C,15938,0.084891,8.489145
A,25,0.080000,8.000000
B,5014,0.070602,7.060231
D,349,0.060172,6.017192


**Вывод:** По результатам данной сводной таблицы нельзя проследить чёткую зависимость количества должников от их заработка. Однако исключив из списка малочисленные категории всё-таки можно проследить зависимость, в которой становится чётко видно, что категория "С", имеющая средний заработок, менее платёжеспособна чем клиенты с большим уровнем достатка в категории "B". Поэтому можно сделать вывод о более выгодном сотрудничестве с клиентами, имеющими высокий уровень достатка, что, в принципе, логично.

Создадим сводную таблицу для решения нашей задачи:

In [28]:
data_purpose = data.pivot_table(index = 'purpose_category',values = 'debt',aggfunc = ['count','mean'])

data_purpose['debt_pers'] = data_purpose['mean']['debt']*100
data_purpose = data_purpose.sort_values('debt_pers',ascending=False)
display(data_purpose)

# debt_pers демонстрирует процент должников в зависимости от цели  кредита

,count,mean,debt_pers
,debt,debt,
purpose_category,,,
операции с автомобилем,4281,0.093436,9.343611
получение образования,3989,0.092504,9.250439
проведение свадьбы,2324,0.078744,7.874355
операции с недвижимостью,10754,0.072531,7.253115


**Вывод:** В данной сводной таблице можно проследить, что клиенты, берущие кредит на операции с недвижимостью более ответственно подходят к выплате долга. Думаю, это связано с тем, что данные цели имеют различные средние значения суммы для кредита, поэтому операции с недвижимостью имеют меньший процент должников, так как, например, ипотека на первую квартику имеет куда большую важность для людей, чем покупка автомобиля. Соответственно люди выплачивают данные кредиты лучше. Хоть разница, как и предыдущих случаях невелика, но в условиях бизнеса это является весомым доводом к определенным действиям со стороны банка.

## Общий вывод.

В ходе данного проекта была проведена **предобработка данных**:

* Заполнение пропусков
* Смена типа данных
* Удаление дубликатов
* Категоризация данных 

В результате мы смогли создать корректные сводные таблицы в зависимости от следующих **показателей**:

* Количество детей
* Семейное положение
* Уровень дохода
* Цель кредита

В итоге мы смогли определить, что более низкий уровень задолженностей имеют клиенты, не имеющие детей, находящиеся в браке, имеющие доход от 200 000 до 1 000 000 рублей, а также берущие кредит для недвижимости, в то время как остальные категории людей имеют на 2,5 - 3% выше задолженность, что в реалиях бизнеса имеет высокую значимость при принятии решений в сторону того или иного клиента.